## OCR/Read

#### [Azure Document Intelligence Read 설명 문서](https://learn.microsoft.com/ko-kr/azure/ai-services/document-intelligence/prebuilt/read?view=doc-intel-4.0.0&tabs=sample-code)

-   Optical Character Recognition(OCR) 기능으로 줄, 단어, 위치 및 언어를 감지 Document Intelligence의 다른 모델은 이 모델을 사용하여 이미지나 문서에서 텍스트를 추출


In [28]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest
import numpy as np


import os

# 환경변수 설정
endpoint = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
key = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY")

# 클라이언트 초기화
document_intelligence_client = DocumentIntelligenceClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

print("Azure Document Intelligence 클라이언트 초기화 완료")

Azure Document Intelligence 클라이언트 초기화 완료


In [29]:
def format_bounding_box(bounding_box):
    if not bounding_box:
        return "N/A"
    reshaped_bounding_box = np.array(bounding_box).reshape(-1, 2)
    return ", ".join(["[{}, {}]".format(x, y) for x, y in reshaped_bounding_box])


def analyze_read():
    url = (
        "https://blog.kakaocdn.net/dn/I43qo/btrwXzcUPWy/niBhpxVT0AUtgKkRwQKDcK/img.jpg"
    )

    # OCR 분석 실행
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-read", AnalyzeDocumentRequest(url_source=url)
    )
    result = poller.result()

In [30]:
# 전체 결과 세부 정보 출력

print("=== 문서 분석 결과 세부 정보 ===")

for idx, style in enumerate(result.styles):
    print(
        f"문서에는 {'필기체' if style.is_handwritten else '인쇄체'} 내용이 포함되어 있습니다."
    )

for page in result.pages:
    print(f"\n--- 페이지 #{page.page_number} 분석 ---")
    print(f"페이지 크기: 너비 {page.width}, 높이 {page.height}, 단위: {page.unit}")

    print("\n텍스트 라인:")
    for i, line in enumerate(page.lines[:10]):  # 처음 10개 라인만 표시
        print(
            f"  라인 #{i}: '{line.content}' (위치: {format_bounding_box(line.polygon)})"
        )

    if len(page.lines) > 10:
        print(f"  ... 그 외 {len(page.lines) - 10}개 라인")

    print("\n단어 신뢰도:")
    for i, word in enumerate(page.words[:20]):  # 처음 20개 단어만 표시
        print(f"  단어: '{word.content}', 신뢰도: {word.confidence:.4f}")

    if len(page.words) > 20:
        print(f"  ... 그 외 {len(page.words) - 20}개 단어")

=== 문서 분석 결과 세부 정보 ===

--- 페이지 #1 분석 ---
페이지 크기: 너비 3024.0, 높이 3024.0, 단위: LengthUnit.PIXEL

텍스트 라인:
  라인 #0: 'K20929 EXP' (위치: [886.0, 306.0], [1215.0, 309.0], [1214.0, 360.0], [886.0, 356.0])
  라인 #1: '20241115까지' (위치: [887.0, 360.0], [1273.0, 365.0], [1272.0, 418.0], [886.0, 412.0])
  라인 #2: '성원을 0' (위치: [950.0, 598.0], [1063.0, 576.0], [1068.0, 603.0], [957.0, 634.0])
  라인 #3: '局易号' (위치: [996.0, 615.0], [954.0, 672.0], [927.0, 657.0], [968.0, 595.0])
  라인 #4: '10' (위치: [1364.0, 613.0], [1510.0, 624.0], [1510.0, 723.0], [1364.0, 733.0])
  라인 #5: '피부과' (위치: [1798.0, 625.0], [1894.0, 627.0], [1893.0, 665.0], [1798.0, 661.0])
  라인 #6: '테스트' (위치: [1792.0, 668.0], [1892.0, 666.0], [1891.0, 704.0], [1792.0, 705.0])
  라인 #7: '완료' (위치: [1811.0, 713.0], [1886.0, 713.0], [1885.0, 747.0], [1811.0, 747.0])
  라인 #8: '일리윤 세라마이드 무향 립밤' (위치: [1091.0, 877.0], [1775.0, 885.0], [1774.0, 950.0], [1090.0, 942.0])
  라인 #9: '내 입술에 직접 닿는 립밤 성분이 중요!' (위치: [1055.0, 1016.0], [1815.0, 1024.0], [1814.0, 1084.0

: 